In [1]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/liputan6-clean-dataset/final_valid_set.csv
/kaggle/input/liputan6-clean-dataset/final_train_set.csv
/kaggle/input/liputan6-clean-dataset/final_test_set.csv


In [2]:
import os
import json
import pprint
import re
import pickle
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import json

In [3]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [ ]:
!pip install datasets evaluate rouge_score transformers[torch] accelerate -U

In [4]:
train_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_train_set.csv')
valid_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_valid_set.csv')
test_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_test_set.csv')

In [5]:
train_set

,final_clean_article,combined_clean_summary
0,Kolonel Richard Ginting dinonaktifkan dari jab...,Kolonel Richard Ginting dinyatakan terlibat bi...
1,Acong ditangkap polisi karena mendalangi pencu...,Acong dipergoki rekan sesama satpam membawa ka...
2,Presiden Susilo Bambang Yudhoyono menggelar ja...,Jamuan makan malam bagi peserta KAA diadakan d...
3,Lebih dari seratus pengikut aliran Falun Gong ...,Mereka berdemonstrasi di saat para delegasi me...
4,Mantan Deputi Kepala Badan Penyehatan Perbanka...,"TMM, mantan Deputi Kepala Badan Penyehatan Per..."
...,...,...
9995,Presiden Susilo Bambang Yudhoyono telah meneke...,Presiden Susilo Bambang Yudhoyono telah menand...
9996,Sejumlah mahasiswa asal Maluku menggelar unjuk...,Sejumlah mahasiswa dari Aliansi Gerakan Peduli...
9997,"Dua kelompok massa, Front Perjuangan Pemuda In...",Massa Front Perjuangan Pemuda Indonesia (FPPI)...
9998,Wanita identik dengan keindahan dan kelembutan...,Sebanyak delapan pelukis wanita ikut serta dal...


In [6]:
valid_set

,final_clean_article,combined_clean_summary
0,Pemerintah masih memberikan waktu dua minggu l...,Pemerintah memberikan tenggat 14 hari kepada p...
1,Diperkirakan 11 juta penduduk Indonesia positi...,Satu dari 20 orang Indonesia diperkirakan meng...
2,Aksi peledakan kantor pemerintah di Aceh terny...,Peledakan bom kembali terjadi di Aceh. Kali in...
3,Petugas Kepolisian Resor Surabaya Timur menemb...,Polres Surabaya Timur menangkap seorang penged...
4,Yogyakarta dan Bali memang gudangnya para seni...,Lima seniman Yogyakarta dan Bali menggelar pam...
...,...,...
10967,Sembilan truk yang memuat kayu balok tanpa dil...,"Pencurian kayu di Palu, Sulawesi Tengah, semak..."
10968,Dua nelayan tewas dan tiga lainnya luka-luka d...,Seorang anggota TNI Angkatan Laut menembaki ka...
10969,Kepolisian Resor Jakarta Pusat menangkap Indri...,"Indriadi, warga Kampung Tarif, Semarang, Jawa ..."
10970,Pemerintah Cina menjanjikan akses pasar yang l...,Pemerintah Cina menjanjikan akses pasar yang l...


In [7]:
test_set

,final_clean_article,combined_clean_summary
0,Kepolisian Daerah Riau bertekad memberantas pe...,Kapolda Riau baru Brigjen Pol. Johny Yodjana b...
1,Bank Indonesia dinilai masih akan menghadapi s...,Kendati Bank Sentral AS menurunkan suku bungan...
2,Berbagai kendala menghambat pendapatan negara ...,Pemerintah bermaksud akan lebih mengandalkan s...
3,Penghapusan beberapa pasal menyangkut hak buru...,"Revisi Kepmennaker Nomor 78 Tahun 2001, dinila..."
4,"Operasi Sadar Jaya yang dilancarkan malam, se...",Polisi menangkap 32 pengunjung Diskotik Mileni...
...,...,...
10967,"Kejaksaan Agung, hingga kini, masih menunggu p...",Kejagung masih menunggu putusan hakim PN Jakse...
10968,Dewan Pimpinan Pusat Partai Kebangkitan Bangsa...,Ketua DPP PKB Khofifah Indarparawansa menuduh ...
10969,"Warga Kampung Melayu, Jakarta Timur, menggelar...","Menyambut Tahun Baru 2002, warga Kampung Melay..."
10970,"Romadhani alias Roban, penjahat kelas kakap, t...",Romadhani alias Roban tewas ditembak polisi sa...


In [8]:
train_set = train_set.sample(n=2000, random_state=42).reset_index(drop=True)
valid_set = valid_set.sample(n=1000, random_state=42).reset_index(drop=True)
test_set = test_set.sample(n=1000, random_state=42).reset_index(drop=True)

In [9]:
train_set

,final_clean_article,combined_clean_summary
0,"Suasana aksi unjuk rasa di depan Gedung DPR, J...","Aksi demo di depan gedung DPR, Senayan Jakarta..."
1,KABUT duka menyelimuti presenter dan bintang s...,KABUT duka menyelimuti presenter dan bintang s...
2,Lokasi wisata Taman Ade Irma Suryani di Cirebo...,Letaknya yang dekat dengan Pelabuhan Cirebon m...
3,TAK ada yang janggal dari penampilan duo RATU ...,TAK ada yang janggal dari penampilan duo RATU ...
4,Keganasan Liverpool kala menggulung dua tim ra...,"Winger Liverpool asal Spanyol, Albert Riera, m..."
...,...,...
1995,Liputan6. com: Bandung: Dua rumah di aliran su...,"Dua rumah di aliran sungai Cikapundung, Bandun..."
1996,Sekitar 100 sopir dan kenek angkutan kota juru...,Ratusan sopir angkutan perkotaan jurusan Karan...
1997,Tabung pengecoran logam di sebuah rumah di Cir...,"Karena baut lepas, tabung pengecoran logam di ..."
1998,Berbagai kunjungan luar negeri yang kerap dila...,Kunjungan luar negeri yang kerap dilakukan Pre...


In [10]:
valid_set

,final_clean_article,combined_clean_summary
0,Satuan Tugas Timor yang dibentuk sejak pekan s...,Restrukturisasi PT Timor Putra Nasional dihara...
1,"Kondisi Taman Nasional Gunung Leuser, Medan, S...",Seluas 9. 000 hektare lahan di Taman Nasional ...
2,Antraks bukan penyakit yang aneh. Penyakit ter...,Kepala Dinas Peternakan DKI Edy Setiarto meneg...
3,Wakil Presiden Megawati Sukarnoputri berharap ...,Wapres Megawati Sukarnoputri mencanangkan akse...
4,Pemerintah Daerah Yogyakarta menolak pengurang...,"Besar kemungkinan, dana alokasi umum untuk dae..."
...,...,...
995,Kepolisian Daerah Metro Jaya dan Komando Daera...,Kapolda Metro dan Pangdam Jaya akan menggelar ...
996,Rencana pemerintah membangun rumah sederhana d...,Rencana pemerintah membangun RS dan RSS belum ...
997,Hukum belum ditegakkan pada dunia lingkungan h...,Jumlah perusakan dan pencemaran lingkungan hid...
998,Tim reserse Kepolisian Resor Kota Kesatuan Pen...,Polres Surabaya menggagalkan pengiriman empat ...


In [11]:
test_set

,final_clean_article,combined_clean_summary
0,"Pengamat politik Andi Malarangeng, baru-baru i...",Kode etik DPR dinilai tak akan berarti banyak ...
1,Pemerintah harus menyelesaikan Kasus Aceh seca...,Penuntasan Kasus Aceh harus konsisten dan komp...
2,Pemerintah terus berupaya memprakarsai islah n...,Pemerintah terus berupaya memprakarsai islah n...
3,Tim Penyidik Kejaksaan Agung memeriksa bekas D...,Bekas Direktur Umum PT Musi Hutan Persada Paul...
4,Kenaikan harga bahan bakar minyak masih berbun...,Para sopir angkutan kota di Bengkulu menolak k...
...,...,...
995,Kenaikan harga bahan bakar minyak tentu berpen...,Meski peraturan resmi kenaikan tarif angkutan ...
996,Presiden Megawati Sukarnoputri mengajak segena...,Presiden Megawati menghadiri peringatan Hari K...
997,Sejumlah anggota Dewan Pimpinan Pusat Partai A...,Menyusul perusakan mobil dinas Ketua MPR Amien...
998,"Menteri Koordinator Bidang Politik, Sosial, da...","Untuk mengakhiri kemelut di tubuh Polri, tongk..."


In [12]:
train_set.shape, valid_set.shape, test_set.shape

((2000, 2), (1000, 2), (1000, 2))

In [13]:
train_set.duplicated().sum(), valid_set.duplicated().sum(), test_set.duplicated().sum()

(0, 0, 0)

In [ ]:
pip install --upgrade pyarrow datasets


In [14]:
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EncoderDecoderModel
import numpy as np

train_dataset = Dataset.from_pandas(train_set)
valid_dataset = Dataset.from_pandas(valid_set)
test_dataset = Dataset.from_pandas(test_set)

2024-07-09 14:24:55.654212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 14:24:55.654350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 14:24:55.795752: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [16]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 1000
    })
})

In [17]:
train_dataset = dataset_dict['train']
print(train_dataset[0])

{'final_clean_article': 'Suasana aksi unjuk rasa di depan Gedung DPR, Jalan Gatot Subroto, Senayan, Jakarta,  makin memanas. Massa membongkar paksa pagar tol dalam kota sepanjang 10 meter. Kendati demikian, aksi ini tidak sempat mengganggu arus lalu lintas tol dari arah Semanggi menuju Grogol, maupun sebaliknya. Sebelumnya, massa sempat memblokade jalan tol, namun segera digagalkan polisi, meski sempat terjadi bentrok. Aksi ini berawal dari kekecewaan pendemo yang diusir dengan penyemprot air atau water cannon, karena berusaha mendekati gerbang Gedung DPR. Pendemo yang dihujani air tak mampu membalasnya, sehingga merusak pagar jalan tol. Mereka bahkan sempat menyandera kendaraan besar, sehingga sempat terjadi kemacetan. Untuk menghindari kemacetan parah, polisi menangkap pendemo yang diduga provokator. Untuk mengantisipasi agar massa tidak sempat masuk ke dalam halaman gedung DPR, polisi kembali melakukan barikade langsung di depan pintu masuk. (ETA).', 'combined_clean_summary': 'Aksi 

In [18]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel, EncoderDecoderModel, BertTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import evaluate
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np

In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("panggi/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("panggi/t5-base-indonesian-summarization-cased")

tokenizer_config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [20]:
prefix = "summary: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["final_clean_article"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    labels = tokenizer(text_target=examples["combined_clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [23]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
rouge = evaluate.load("rouge")

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./fine_tune_T5_model_for_Summarization",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir='./logs',
    fp16=True,
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.290500,2.660467,0.298300,0.151500,0.254100,0.254200,18.597000
2,2.008700,2.615897,0.304200,0.154700,0.258200,0.258300,18.972000
3,1.960200,2.598628,0.307800,0.157500,0.261600,0.261900,18.996000
4,2.044500,2.592332,0.311500,0.159900,0.264700,0.264600,18.995000
5,1.761200,2.594293,0.311500,0.160400,0.265000,0.265000,18.996000
6,1.791600,2.595874,0.312200,0.160800,0.266200,0.266300,18.996000
7,1.840600,2.594591,0.313300,0.162100,0.267500,0.267500,18.996000
8,1.868000,2.594319,0.312700,0.162000,0.267200,0.267200,18.996000
9,1.752400,2.591735,0.312600,0.161900,0.267000,0.267000,18.996000
10,1.766000,2.590694,0.313400,0.162200,0.267800,0.268000,18.996000


TrainOutput(global_step=1250, training_loss=1.8797090438842774, metrics={'train_runtime': 1830.7445, 'train_samples_per_second': 10.925, 'train_steps_per_second': 0.683, 'total_flos': 6089521815797760.0, 'train_loss': 1.8797090438842774, 'epoch': 10.0})

In [27]:
# Text to summarize
ARTICLE_TO_SUMMARIZE = """Liputan6.com, Jakarta Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC. Fungsi dari Kartu Debit Mandiri ini sangat beragam, mulai dari tarik tunai, setor tunai, transfer uang, cek saldo rekening, hingga membayar berbagai tagihan melalui mesin ATM.
Penting bagi Anda yang ingin membuka rekening tabungan di Bank Mandiri untuk memahami jenis Kartu Debit Mandiri agar tidak salah memilih. Setiap kartu debit Mandiri memiliki kelebihan dan kekurangannya masing-masing, sehingga penting bagi nasabah untuk memilih yang sesuai dengan kebutuhan dan preferensi mereka.
Dalam memilih jenis Kartu Debit Mandiri, nasabah perlu mempertimbangkan kebutuhan dan gaya hidup mereka. Apakah mereka membutuhkan manfaat tambahan seperti asuransi atau akses ke airport lounge, ataukah mereka menginginkan kartu debit yang sederhana namun praktis. Dengan mengetahui jenis Kartu Debit Mandiri yang sesuai, nasabah dapat memaksimalkan manfaat yang mereka dapatkan dari penggunaan kartu debit tersebut.
"""

# Move model to GPU
model.to('cuda')

# Measure the time for prediction
start_time = time.time()

# Generate summary
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt').to('cuda') 
summary_ids = model.generate(input_ids,
            max_length=100, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True)

end_time = time.time()

# Decode and print the summary
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

# Print the time taken
print(f"Time taken for prediction: {end_time - start_time:.2f} seconds")

Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC.
Time taken for prediction: 1.00 seconds


In [32]:
# Text to summarize
ARTICLE_TO_SUMMARIZE = """Liputan6.com, Jakarta Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC. Fungsi dari Kartu Debit Mandiri ini sangat beragam, mulai dari tarik tunai, setor tunai, transfer uang, cek saldo rekening, hingga membayar berbagai tagihan melalui mesin ATM.
Penting bagi Anda yang ingin membuka rekening tabungan di Bank Mandiri untuk memahami jenis Kartu Debit Mandiri agar tidak salah memilih. Setiap kartu debit Mandiri memiliki kelebihan dan kekurangannya masing-masing, sehingga penting bagi nasabah untuk memilih yang sesuai dengan kebutuhan dan preferensi mereka.
Dalam memilih jenis Kartu Debit Mandiri, nasabah perlu mempertimbangkan kebutuhan dan gaya hidup mereka. Apakah mereka membutuhkan manfaat tambahan seperti asuransi atau akses ke airport lounge, ataukah mereka menginginkan kartu debit yang sederhana namun praktis. Dengan mengetahui jenis Kartu Debit Mandiri yang sesuai, nasabah dapat memaksimalkan manfaat yang mereka dapatkan dari penggunaan kartu debit tersebut.
"""

# model to CPU
model.to('cpu')

# Measure the time for prediction
start_time = time.time()

# Generate summary
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt').to('cpu') 
summary_ids = model.generate(input_ids,
            max_length=100, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True)

end_time = time.time()

# Decode and print the summary
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

# Print the time taken
print(f"Time taken for prediction: {end_time - start_time:.2f} seconds")


Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC.
Time taken for prediction: 2.51 seconds
